In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.manifold import TSNE
from ipywidgets import interactive, HBox, VBox
from sklearn.cluster import KMeans

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings('ignore')

In [2]:
from plotly.offline import init_notebook_mode, iplot
import plotly.offline as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
primeiroCiC = ['116301', '118001', '118010', '113034', '140481', '145971']
segundoCiC = ['116319', '115045', '118028', '118036', '113042']
terceiroCiC = ['118044', '118052', '113956', '113051', '117366']


primeiroLic1 =['116793', '117366', '145971', '147389', '194221']
segundoLic1 = ['113034', '116301', '125156']
terceiroLic1 = ['115045', '116319', '125172']

primeiroLic2 =['113492', '194221', '113450', '113476']
segundoLic2 = ['113034', '116319', '191027', '150649', '194280']
terceiroLic2 = ['115045', '113093', '117366', '117889']

primeiroEngComp =['113034', '113093', '113476', '117528', '118001', '118010']
segundoEngComp = ['118028', '118036', '113042', '115045', '116319']
terceiroEngComp =['113051', '117242', '170054', '201600', '206075']


primeiroMec = ['114626', '114634', '118010', '113476', '113034', '118001', '168891']
segundoMec = ['168891', '113093', '118028', '118036', '115045', '168874']
terceiroMec = ['113051', '118044', '118052', '168769', '116319']

primeiroRedes = ['113034', '113093', '118001', '118010', '167959']
segundoRedes = ['113042', '115045', '118028', '118036', '169676']
terceiroRedes = ['113051', '113301', '167983', '169749']


undropableCol = ["curso","saiu","Cotista","Raça","Sexo","Tipo de Escola"]#, 'Ingresso_unb'

In [5]:
df0 = pd.read_csv("aluno_quant_Materias_Media_cotistas_0.csv", index_col=0)
df1 = pd.read_csv("aluno_quant_Materias_Media_cotistas_1.csv", index_col=0)
df2 = pd.read_csv("aluno_quant_Materias_Media_nao_cotistas_2.csv", index_col=0)
df3 = pd.read_csv("aluno_quant_Materias_Media_cotistas_3.csv", index_col=0)
df4 = pd.read_csv("aluno_quant_Materias_Media_cotistas_4.csv", index_col=0)
df5 = pd.read_csv("aluno_quant_Materias_Media_cotistas_5.csv", index_col=0)

In [6]:
dfComp = df0[undropableCol+primeiroLic1+segundoLic1+terceiroLic1].fillna(-1)
dfRedes = df1[undropableCol+primeiroRedes+segundoRedes+terceiroRedes].fillna(-1)
dfCic = df2[undropableCol+primeiroCiC+segundoCiC+terceiroCiC].fillna(-1) 
dfMec = df3[undropableCol+primeiroMec+segundoMec+terceiroMec].fillna(-1) 
dfEng = df4[undropableCol+primeiroEngComp+segundoEngComp+terceiroEngComp].fillna(-1) 

In [7]:
def Diff(li1, li2): 
    return (list(set(li1) - set(li2))) 

In [8]:
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

In [72]:
def formatColor(color):
    return 'rgb('+str(int(color[0]*255))+','+str(int(color[1]*255))+','+str(int(color[2]*255))+')'


    
def tsne(df, currentLabel = 'saiu', 
         selectedCols = ["curso","saiu","Cotista","Raça","Sexo","Tipo de Escola","index"],
        n_clusters=[5]):
    dfcopy = df.copy()
    originalMap = df.copy()
    originalMap = originalMap.reset_index()
    df = df.reset_index().drop(['index'], axis=1)
    X = df.values
    X_embedded = TSNE(n_components=2).fit_transform(X)
    
    

    data = df
    Y_sklearn = X_embedded
    y = data[currentLabel].values
    numColors = len(data[currentLabel].unique())

    cmap = get_cmap(numColors+1)
    colorList = [cmap(i) for i in range(1, numColors+1)]


    traceArr = []
    mapPoints = []
    labelArray = data[currentLabel].unique()
    for lab, col in zip(labelArray,
                        colorList):
        mapPoints.append(data[y==lab].index)

        trace1 = go.Scatter(
            x=Y_sklearn[y==lab, 0],
            y=Y_sklearn[y==lab, 1],
            mode='markers',
            name = str(lab),
            marker=dict(
                size=5,
                color=formatColor(col),
                opacity=0.8
            ),

        )
        traceArr.append(trace1)
    f = go.FigureWidget(data=traceArr, layout=go.Layout(
            margin=dict(
                l=0,
                r=0,
                b=0,
                t=0
            )
        ))
    
    t = go.FigureWidget([go.Table(
    header=dict(values=selectedCols,
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[originalMap[col] for col in selectedCols],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))])

    
    def getSelector(index):
        def selection_fn(trace, points, selector):
            
            if (points.trace_index == 0):
                t.data[0].cells.values = [originalMap.loc[mapPoints[index][points.point_inds]][col] for col in selectedCols]
            else:
                w = [originalMap.loc[mapPoints[index][points.point_inds]][col] for col in selectedCols]
                s = t.data[0].cells.values
                w = pd.DataFrame(w).T
                a = pd.DataFrame([pd.Series(x) for x in list(s)]).T
                a.columns = w.columns
                t.data[0].cells.values = a.append(w).T
               
            traceArr = []    
            if points.trace_index == len(labelArray) -1:
                s = t.data[0].cells.values
                a = pd.DataFrame([pd.Series(x) for x in list(s)]).T
                a.columns = selectedCols
                kmeans_values = dfcopy.loc[a['index']].values
                
                kmeans = KMeans(n_clusters=n_clusters[0], random_state=0).fit(kmeans_values)
                
                kmeansMap = pd.DataFrame(data= kmeans.labels_, columns=['label'])
                kmeansMap['id'] =dfcopy.loc[a['index']].index
                
                pointIndex = [originalMap.index[originalMap['index']== ids ].values.tolist() for ids in kmeansMap['id']]
                pointIndex = sum(pointIndex,[])
                kmeansMap['points'] = pointIndex
                
                
                Y_sklearn2 = Y_sklearn[pointIndex]
                y2 = kmeansMap['label'].values
                y3 = kmeansMap['id'].values
                numColors = n_clusters[0]
                
                cmap = get_cmap(numColors+1)
                colorList = [cmap(i) for i in range(1, numColors+1)]
                
                labelArray2 = list(range(n_clusters[0]))
                
                
                for lab, col in zip(labelArray2,
                        colorList):
                    x_cord = Y_sklearn2[y2==lab, 0]
                    y_cord = Y_sklearn2[y2==lab, 1]
                    
                    trace1 = go.Scatter(
                        x=x_cord,
                        y=y_cord,
                        text = y3[y2==lab],
                        hoverinfo = 'text + y + x',
                        mode='markers',
                        name = str(lab),
                        marker=dict(
                            size=5,
                            color=formatColor(col),
                            opacity=0.8
                        ),

                    )
                    traceArr.append(trace1)
                
                g2 = go.FigureWidget(data=traceArr, layout=go.Layout(
                        margin=dict(
                            l=0,
                            r=0,
                            b=0,
                            t=0
                        )
                    ))
                
                t2 = go.FigureWidget([go.Table(
                    header=dict(values=dfcopy.columns,
                                fill = dict(color='#C2D4FF'),
                                align = ['left'] * 5),
                    cells=dict(values=kmeans.cluster_centers_.T,
                               fill = dict(color='#F5F8FF'),
                               align = ['left'] * 5))])
                
#                 g = g2
                display(g2)
                display(t2)
#             g.data[points.trace_index].x = Y_sklearn[mapPoints[index][points.point_inds], 0]
#             g.data[points.trace_index].y = Y_sklearn[mapPoints[index][points.point_inds], 1]
#                 print(originalMap.loc[mapPoints[index][points.point_inds]])
        return selection_fn
        
    for ind, el in enumerate(f.data):
        el.on_selection(getSelector(ind))
        
    def updateKmeans(N_clusters):
        n_clusters[0] = N_clusters
    slider = interactive(updateKmeans, N_clusters=(1, 8, 1))
    display(VBox((slider,f,t)))
    return f, g

In [74]:
f, g = tsne(dfCic.copy())

FigureWidget({
    'data': [{'hoverinfo': 'text+y+x',
              'marker': {'color': 'rgb(133,255,0)', 'opa…

FigureWidget({
    'data': [{'cells': {'align': [left, left, left, left, left],
                        'fill'…